# n-puzzle problem

In [70]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import functools

In [71]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

## Utility functions

In [72]:
def counter(fn):
    """Simple decorator for counting number of calls"""

    @functools.wraps(fn)
    def helper(*args, **kargs):
        helper.calls += 1
        return fn(*args, **kargs)

    helper.calls = 0
    return helper

In [73]:
class State :
    def __init__(self, matrix: np.ndarray):
        self.matrix = matrix
        self.g = float('inf') #cost root to current
        self.f = float('inf') #cost root to goal
        self.h = float('inf') #heuristic : estimated cost from current to goal

    #check if the current state contains a matrix that represents the solution
    def is_goal(self):
        return np.array_equal(self.matrix, np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))
   
    def is_solvable(self) -> bool:
        inv_count = 0
        tmp_state = self.matrix.copy().reshape(PUZZLE_DIM**2)

        # Count inversions
        for i in range(0, PUZZLE_DIM**2):
            for j in range(i + 1, PUZZLE_DIM**2):
                if tmp_state[j] != 0 and tmp_state[i] != 0 and tmp_state[i] > tmp_state[j]:
                    inv_count += 1

        if PUZZLE_DIM % 2 != 0:  # Odd grid size
            return inv_count % 2 == 0
        else:  # Even grid size
            blank_row = PUZZLE_DIM - np.where(self.matrix == 0)[0][0] # Row of blank (from bottom)
        return (inv_count + blank_row) % 2 != 0

In [ ]:
def manhattan_heuristic(state:State,goal_state:State)->int:
    manhattan = 0
    for elem in range(1,PUZZLE_DIM**2):
        start_coords = np.where(state.matrix == elem)
        end_coords = np.where(goal_state.matrix == elem)
        manhattan += abs(start_coords[0]-end_coords[0])+abs(start_coords[1]-end_coords[1])
    return manhattan

def available_actions(state: State) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state.matrix == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

@counter
def do_action(state: State, action: 'Action') -> State:
    new_state = state.matrix.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return State(new_state)

def print_solution(solution : tuple[list,int]):
    if solution[0] == None:
        print("No solution was found")
    else:
        print(f"Solution was found in {len(solution[0])} steps with cost of {do_action.calls}")
        print("Solution path:")
        for state in solution[0]:
            print(state.matrix)
            print()

## IDA*

In [ ]:
# takes as parameter a state and a heuristic function
def ida_star(initial_state : State, goal_state : State, heuristic):
    bound = heuristic(initial_state,goal_state)
    path = [initial_state]

    while True:
        t = search(path, 0, bound, heuristic, goal_state, set())
        if t < 0: ## FOUND = -1
            return (path, bound)
        elif t == float('inf'):
            return (None, float('inf')) 
        else: # t belongs to [0,inf[
            bound = t

def search(path : list, g : float, bound : int, h, goal_state : State, visited = set()) :
    node = path[-1]
    f = g + h(node, goal_state)
    if f > bound :
        return f
    if node.is_goal():
        return -1 # FOUND
    
    min = float('inf')
    visited.add(node)

    successors = [do_action(node, action) for action in available_actions(node)]

    for successor in sorted(successors, key=lambda s: h(s, goal_state)):
        if successor not in visited:
            path.append(successor)
            t = search(path, g + 1, bound, h, goal_state, visited)
            if t == -1:
                return -1
            if t < min :
                min = t
            path.pop()

    visited.remove(node) #backtrack
    return min

### Hard coded matrices to be used for testing

In [ ]:
""" state = State(np.array([np.array([1,8,2]),np.array([0,4,3]),np.array([7,6,5])]))
state = State(np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])]))
print(state.is_solvable())

### complex 5x5 test
state = np.array([[13,  9,  0, 10, 19],
                [ 3, 21, 14,  5,  8],
                [22, 16,  4, 24, 18],
                [ 6,  2, 11,  1, 20],
                [ 7, 15,23,12,17]])

state = State(state)
print(state.is_solvable())
print(is_solvable(state.matrix))

###  4x4 tests
state = np.array([[14,  13,  15, 8],
                [ 1, 4 , 12, 6],
                [2, 7,  10, 3],
                [ 11,  5, 0,  9]])

state = State(state)
print(state.is_solvable())
state = np.array([[12,  1,  2, 15],
                [ 11, 6 , 5,8],
                [7, 10,  9, 4],
                [ 0,  13, 14,  3]])

state = State(state)
print(state.is_solvable()) """

### Find solution for random state

In [77]:
RANDOMIZE_STEPS = 1000
goal_state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))
state = State(np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM)))

for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

state = State(np.array([np.array([0,2,1]),np.array([3,7,5]),np.array([8,6,4])]))
print(state.is_solvable())
print(state.matrix)

if state.is_solvable():
    solution = ida_star(state,goal_state, manhattan_heuristic)
    print_solution(solution)
else:
    print("not Solvable")

Randomizing:   0%|          | 0/1000 [00:00<?, ?it/s]

True
[[0 2 1]
 [3 7 5]
 [8 6 4]]
Solution was found in 23 steps with cost of 216080
Solution path:
[[0 2 1]
 [3 7 5]
 [8 6 4]]

[[3 2 1]
 [0 7 5]
 [8 6 4]]

[[3 2 1]
 [7 0 5]
 [8 6 4]]

[[3 0 1]
 [7 2 5]
 [8 6 4]]

[[3 1 0]
 [7 2 5]
 [8 6 4]]

[[3 1 5]
 [7 2 0]
 [8 6 4]]

[[3 1 5]
 [7 2 4]
 [8 6 0]]

[[3 1 5]
 [7 2 4]
 [8 0 6]]

[[3 1 5]
 [7 2 4]
 [0 8 6]]

[[3 1 5]
 [0 2 4]
 [7 8 6]]

[[3 1 5]
 [2 0 4]
 [7 8 6]]

[[3 0 5]
 [2 1 4]
 [7 8 6]]

[[0 3 5]
 [2 1 4]
 [7 8 6]]

[[2 3 5]
 [0 1 4]
 [7 8 6]]

[[2 3 5]
 [1 0 4]
 [7 8 6]]

[[2 3 5]
 [1 4 0]
 [7 8 6]]

[[2 3 0]
 [1 4 5]
 [7 8 6]]

[[2 0 3]
 [1 4 5]
 [7 8 6]]

[[0 2 3]
 [1 4 5]
 [7 8 6]]

[[1 2 3]
 [0 4 5]
 [7 8 6]]

[[1 2 3]
 [4 0 5]
 [7 8 6]]

[[1 2 3]
 [4 5 0]
 [7 8 6]]

[[1 2 3]
 [4 5 6]
 [7 8 0]]

